# Assignment 2 - Di Riccio Tommaso - 14/04/2024

To implement the Bayesian Network, I decided to first create two Python classes, Node and BayesianNetwork, that allow modeling any Bayesian Network with nodes whose distribution is binomial or multinomial and which have discrete values. <br><br>

The Node class represents a node of the Bayesian Network. Its attributes are:
- name: uniquely identifies the node within the network.

- probabilities: stores the conditional probability table of the node. If the node has parents, it is implemented with a dictionary in which the keys are tuples representing parent node configurations, and the values are dictionaries storing the node's values as keys and the associated probabilities as values (e.g. (True, False): {True: 0.4, False: 0.6}). <br>For root nodes, only the inner dictionary is stored (e.g. {True: 0.4, False: 0.6})

- parents: a list maintaining references to parent nodes in the network.

- childs: a list maintaining references to child nodes in the network.
<br>

The most interesting method in this class is <code>sample_node(self, parents_values = [])</code> which allows sampling a value for the random variable. Using the parent values passed as an argument (if any), it accesses the correct probability distribution stored in the object, and uses the helper method <code>sample_value(self, values, probabilities)</code> to return one of the possible node values according to this distribution.

In [ ]:
import random
from typing import List, Dict, Tuple, Any

class Node:
    def __init__(self, name: str, probabilities: Dict[Tuple[Any], Any] = {}, parents: List['Node'] = []):
        self.name = name
        self.probabilities = probabilities
        self.parents = parents
        self.childs = []
        # Add this node as a child to its parents
        for parent in parents:
            parent.add_child(self)

    # Get the name of the node
    def get_name(self) -> str:
        return self.name

    # Add a child to the node
    def add_child(self, child: 'Node') -> None:
        if child not in self.childs:
            self.childs.append(child)

    # Get the parents of the node
    def get_parents(self) -> List['Node']:
        return self.parents

    # Get the childs of the node
    def get_childs(self) -> List['Node']:
        return self.childs

    # Get all the probabilities of the node values
    def get_probabilities(self) -> Dict[Any, Any]:
        return self.probabilities

    # Get the probability of a node value given the parents values
    def get_probability(self, node_value: Any, parents_values: Tuple[Any] = []) -> float:
        # If the node has no parents, return the probability of the node_value
        if self.parents == []:
            return self.probabilities.get(node_value, None)
        # If the node has parents, return the probability of the node_value given the parents_values
        values = self.probabilities.get(parents_values, None)
        if values:
            return values.get(node_value, None)
        return None

    # Sample a value for the node
    def sample_node(self, parents_values: Tuple[Any] = []) -> Any:
        if self.parents == []:
            values = self.probabilities.keys()
            probabilities = self.probabilities.values()
            return self.__sample_value(values, probabilities)
        else:
            values = self.probabilities.get(parents_values).keys()
            probabilities = self.probabilities.get(parents_values).values()
            return self.__sample_value(values, probabilities)

    def __sample_value(self, values, probabilities) -> Any:
        # Generate a random number between 0 and 1
        rand_num = random.random()
        # Initialize cumulative probability
        cumulative_prob = 0
        # Iterate through values and probabilities to find the sampled value
        for value, prob in zip(values, probabilities):
            cumulative_prob += prob
            if rand_num <= cumulative_prob:
                return value
        # Note: an easy alternative is to use numpy.random.choice

The class BayesianNetwork is used to construct and manage the Bayesian Network. Its attributes are
- nodes: a list storing the collection of nodes that constitute the Bayesian Network.
- isSorted: indicates whether the nodes are currently topologically sorted, used to optimize ancestral sampling.

The basic operations implemented by this class include adding and removing nodes from the network. Additionally, the <code>is_valid(self)</code> method checks if the network is well-defined by verifying that the probability distributions of the nodes sum to one and that all the required nodes are in the network.<br><br>
The <code>ancestral_sampling(self)</code> method allows sampling of the entire network. If necessary, it uses the <code>topological_sort(self)</code> method to sort the nodes in topological order, using a recursive depth-first exploration of the tree and reversing the resulting node list. Then it calls <code>sample_node(self, parents_values)</code> on each node. The data structure used during the process is a dictionary with the nodes name as keys and the result of the sampling as values. When all the nodes have been sampled, this dictionary is returned.

In [ ]:
from typing import List, Optional

class BayesianNetwork:
    def __init__(self, nodes: List['Node'] = []):
        self.nodes = nodes
        self.isSorted = False

    # Add a node to the network
    def add_node(self, node: 'Node') -> None:
        # Check if name is unique
        if self.get_node_by_name(node.name):
            raise ValueError(f"Node with name {node.name} already exists")
        self.isSorted = False
        self.nodes.append(node)

    # Remove a node from the network
    def remove_node(self, node: 'Node') -> None:
        self.nodes.remove(node)

    # Get a node by its name
    def get_node_by_name(self, name: str) -> Optional['Node']:
        for node in self.nodes:
            if node.name == name:
                return node
        return None

    # Check if the network is valid
    def is_valid(self) -> bool:
        for node in self.nodes:
            # Check if all probabilities sum to 1
            probabilities = node.get_probabilities()
            if node.get_parents():
                for key in probabilities.keys():
                    if round(sum(probabilities[key].values()), 2) != 1:
                            return False
            else:
                if round(sum(probabilities.values()), 2) != 1:
                    return False

            # Check if all parents are in the network
            for parent in node.get_parents():
                if parent not in self.nodes:
                    return False

        return True

    # Ancestral sampling
    def ancestral_sampling(self) -> dict[str, float]:

        # Topological sort
        if not self.isSorted:
            self.topological_sort()

        # Sample the nodes
        node_values = {}
        for node in self.nodes:
            if not node.get_parents():
                node_values[node.get_name()] = node.sample_node()
            else:
                parents_values = tuple([node_values[parent.get_name()] for parent in node.get_parents()])
                node_values[node.get_name()] = node.sample_node(parents_values)

        return node_values

    # Topological sort of the network
    def topological_sort(self) -> None:
        sorted_nodes = []
        visited = set()
        for node in self.nodes:
            self.__visit(node, visited, sorted_nodes)
        self.nodes = sorted_nodes[::-1]
        self.isSorted = True

    def __visit(self, node: 'Node', visited: set, sorted_nodes: List['Node']) -> None:
        if node in visited:
            return
        visited.add(node)
        for child in node.get_childs():
            self.__visit(child, visited, sorted_nodes)
        sorted_nodes.append(node)

The Bayesian Network I decided to implement is a slightly modified version of the one proposed by the University of British Columbia for a case study on causes of student deaths ([here at pag. 5](https://wiki.ubc.ca/images/6/68/Bayesian_Networks.pdf#page=5)). In particular, two nodes have been removed and a multinomial distribution has been added. The structure of this version is shown in the following figure. <br></br>

<img src="https://drive.google.com/uc?export=view&id=1isCCrQXLiyXbFTssKZZ_95DNnM1uqzXw" alt="Student’s Death Diagnosis Bayesian Network Model" title="Student’s Death Diagnosis Bayesian Network Model" width=60% height: auto/>

The following code illustrates the creation this network, the realization of a network validity check and the execution of the Ancestral Sampling process through <code>calculate_percentage(iterations, bayesianNetwork)</code>. This method performs <code>iterations</code> sampling procedures on the network, returning, for each node, the percentage of times a certain value appears in the sampled configurations of node values.

In [ ]:
def diagnosis_example():

    bayesianNetwork = BayesianNetwork()

    # Create nodes without parents
    bu = Node("Break_Up", probabilities={True: 0.2, False: 0.8})
    rl = Node("Relative_Lost", probabilities={True: 0.1, False: 0.9})
    nff = Node("No_Friends_And_Family", probabilities={True: 0.5, False: 0.5})
    fnp = Node("Financial_Problems", probabilities={True: 0.3, False: 0.7})
    wsp = Node("Work_And_Study_Pressure", probabilities={True: 0.3, False: 0.7})

    # Create nodes with parents
    es = Node(
        name = "Emotional_Shock",
        probabilities={
            (True, True): {True: 0.85, False: 0.15},
            (True, False): {True: 0.4, False: 0.6},
            (False, True): {True: 0.6, False: 0.4},
            (False, False): {True: 0.05, False: 0.95},
        },
        parents=[bu, rl]
    )

    cst = Node(
        name = "Chronic_Stress",
        probabilities={
            (True, True): {True: 0.95, False: 0.05},
            (True, False): {True: 0.8, False: 0.2},
            (False, True): {True: 0.6, False: 0.4},
            (False, False): {True: 0.1, False: 0.9},
        },
        parents=[fnp, wsp]
    )

    ins = Node(
        name = "Insomnia",
        probabilities={
            (True,): {"Severe": 0.6, "Mild": 0.3, "None": 0.1},
            (False,): {"Severe": 0.05, "Mild": 0.2, "None": 0.75},
        },
        parents=[cst]
    )

    dep = Node(
        name = "Depression",
        probabilities={
            (True, True): {True: 0.98, False: 0.02},
            (True, False): {True: 0.75, False: 0.25},
            (False, True): {True: 0.8, False: 0.2},
            (False, False): {True: 0.1, False: 0.9},
        },
        parents=[es, nff]
    )

    su = Node(
        name = "Suicide",
        probabilities={
            (True, True): {True: 0.8, False: 0.2},
            (True, False): {True: 0.65, False: 0.35},
            (False, True): {True: 0.3, False: 0.7},
            (False, False): {True: 0.05, False: 0.95},
        },
        parents=[dep, cst]
    )

    # Add nodes to network
    bayesianNetwork.add_node(bu)
    bayesianNetwork.add_node(rl)
    bayesianNetwork.add_node(nff)
    bayesianNetwork.add_node(fnp)
    bayesianNetwork.add_node(wsp)
    bayesianNetwork.add_node(es)
    bayesianNetwork.add_node(cst)
    bayesianNetwork.add_node(ins)
    bayesianNetwork.add_node(dep)
    bayesianNetwork.add_node(su)

    # Check network
    if(not bayesianNetwork.is_valid()):
        print("Network is not valid")
        return

    # Sample process
    results = calculate_percentage(1000000, bayesianNetwork)

    for node in results:
        print (node)
        for value in results[node]:
            print ('\t',value,':', results[node][value])


def calculate_percentage(iterations, bayesianNetwork):
    counts = {}
    for _ in range(iterations):
        data = bayesianNetwork.ancestral_sampling()
        for key, value in data.items():
            if key not in counts:
                counts[key] = {}
            if value not in counts[key]:
                counts[key][value] = 0
            counts[key][value] += 1

    percentages = {}
    total_iterations = iterations
    for key, value_counts in counts.items():
        percentages[key] = {}
        for value, count in value_counts.items():
            percentages[key][value] = round((count / total_iterations) * 100, 2)

    return percentages


if __name__ == "__main__":
    diagnosis_example()

Work_And_Study_Pressure
	 True : 29.96
	 False : 70.04
Financial_Problems
	 True : 29.98
	 False : 70.03
Chronic_Stress
	 True : 42.76
	 False : 57.24
Insomnia
	 Mild : 24.34
	 None : 47.18
	 Severe : 28.49
No_Friends_And_Family
	 False : 50.01
	 True : 49.99
Relative_Lost
	 False : 90.0
	 True : 10.0
Break_Up
	 False : 80.01
	 True : 19.99
Emotional_Shock
	 False : 82.72
	 True : 17.28
Depression
	 False : 47.85
	 True : 52.15
Suicide
	 False : 55.2
	 True : 44.8


These are the results from one million episodes of ancestral sampling. To analyze the accuracy of the approximation, we can compare the estimated probability distribution of a random variable, computed with the sampling process, with its real distribution. <br>
To simplify, let's take the random variable "Break up", which is a root node. From its distribution table, we see that the probability of being true is 80%. At the end of the sampling process, it was found to be true 80.01% of the time, demonstrating that one million samples are sufficient to achieve a precise approximation. The same test has been executed on the unique multinomial random variable "Insomnia" with clamped parent values. The results of this test are not shown, but the precision obtained was of the same order of magnitude. <br>
Examing the result we find that, without knowing anything about the student, there is a 44.8% chance that his cause of death is suicide. <br>
Even if the accuracy and correctness of the result was not the primary focus of this proposed case study, I would like to point out that this is clearly a too high value as the World Health Organization (WHO) Mortality Database reveals a true rate of 9.1%, and even lower for European countries [as detailed here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1414751/).

## Possible improvements

In this implementation of the Bayesian Network classes it's currently not possible to modify the network after its creation. An improvement could involve adding the capability to modify the CPT of the nodes and the relationships between them. Another enhancement could be incorporating additional checks to ensure the correctness of the network, such as verifying the absence of loops in the relationships between nodes.

Further possible improvements are:
- extending the code to handle continuous variables, in addition to discrete variables
- implement other forms of sampling, for example allowing evidence variables